In [23]:
#Import Libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options as chromeoptions
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json

In [2]:
#Creating webdriver instance as well as options for fullscreen
options = chromeoptions()
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install()))

In [3]:
#getting url through webdriver and open instance of global crypto data
url="https://api.coinlore.net/api/global/"
driver.get(url)

In [4]:
#obtain contents of page
page_source = driver.page_source

In [29]:
#drop all html tags of page to prep for json structure
json_page=page_source.replace("<html><head></head><body>", "").replace("</body></html>", "")

In [39]:
#store json object of current crypto market data
data = json.loads(json_page)